In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import base64
import json
import logging
import os
import re
import smtplib
import sys
import time
import threading
from functools import partial
from itertools import chain
from importlib import reload
from pathlib import Path
from uuid import uuid4
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser

from api.watchdog import specs
from api.email import df_to_html, smtp_server, from_email, to_email_all, to_email_test
from api.watchdog.specs import get_trades, get_targets
from api.watchdog.pricer import Pricer
from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root,
    encode_df_as_image_tag
)

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [4]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-03-13 00:42:41,038 - Authenticating...
2023-03-13 00:42:41,745 - Successfully authenticated


In [5]:
super_clarion(clarion)
runner = Runner(clarion)

In [6]:
### check metrics
pd.concat(clarion.metrics()).to_clipboard()

### Metrics

In [7]:
trades = get_trades(clarion)

In [8]:
targets = get_targets()

In [9]:
clarion_metrics = dict(
    pv = clarion.metric('pv', 'PresentValue', 'Any'),
    local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any'),
    # currency = clarion.metric('currency', 'PayCurrency', 'Any'),
    strike = clarion.metric('strike', 'Strike', 'Any'),
    notional = clarion.metric('notional', 'Notional', 'Any'),
    expiry = clarion.metric('expiry', 'ExpiryDate', 'Any'),
    maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'}),
    start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'}),
    fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates'),
    delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    gamma = clarion.metric('gamma', 'Gamma', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'}),
    price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'}),
    fwd_price_cents = clarion.metric('fwd_price_cents', 'ForwardPrice', 'Rates', {'Unit': 'Cents'}),
    price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'}),
    price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'}),
    price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'}),
    spot = clarion.metric('spot', 'Spot', 'Rates'),
    # vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'}),
    # atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'}),
    atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'}),
    # strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'}),
    strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'}),
    fx_spot = clarion.metric('fx_spot', 'Spot', 'Fx'),
    fx_fwd = clarion.metric('fx_fwd', 'Forward', 'Fx'),
    # carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'}),
)

In [10]:
def highlight_cond(df, props, col):
    d = pd.DataFrame(None, df.index, df.columns)
    d.loc[df[col]] = props
    return d 

In [11]:
def highlight_target(df, props=None):
    d = pd.DataFrame(None, df.index, df.columns)
    for index, row in df.iterrows():
        props = 'background-color:red; color: Gold; font-weight: bold;' if row['breach'] else 'color: DarkOrange; font-weight: bold;'
        d.loc[index, ['target', row['var']]] = props
    return d

In [12]:
def send_watch_email(s1, s2, test=False):
    to_email = to_email_all if not test else to_email_test
    message = MIMEMultipart("alternative")
    message["Subject"] = f'TRADE WATCH - {pd.Timestamp.now(): %d%b%y %H:%M}'
    message["From"] = from_email
    message["To"] = ','.join(to_email)
    # img_tags = [encode_df_as_image_tag(s) for s in [s1, s2]]
    
    html_body = '<br>' + df_to_html(s1) + ('<br>' * 2) + df_to_html(s2)
    html_ = MIMEText(html_body, "html")
    message.attach(html_)
    smtp = smtplib.SMTP(smtp_server)
    # smtp.set_debuglevel(1)
    smtp.sendmail(from_email, to_email, message.as_string())

In [1]:
table_styles = {
    'tr, td': [
      ('white-space', 'nowrap'),
    ],
    'th': [
        ('white-space', 'nowrap')
    ],
    'td': [
        ('white-space', 'nowrap')
    ],
}
table_styles = [{"selector": sel, 'props': '; '.join([': '.join(p) for p in props])} for sel, props in table_styles.items()]
def style_watch(watch):
    style = watch.style.apply(highlight_cond, props='background-color:red; color: white; font-weight: bold;', col='breach', axis=None) \
                .apply(highlight_target, axis=None) \
                .format(na_rep='') \
                .format( 
                    partial(millify, decimal=3, pct=None, zero_rep='-'), 
                    subset=watch.columns[watch.dtypes == 'float64'].drop('target')
                ) \
                .format( 
                    partial(millify, decimal=3, pct=None, zero_rep='0'), 
                    subset='target'
                ) \
                .set_table_styles(table_styles) \
                .set_caption('Trades and triggers')
    return style

def style_details(details):
    style = details.style.format(na_rep='').format(
                partial(millify, decimal=3, pct=None, zero_rep='-'), 
                subset=details.columns[details.dtypes == 'float64']
            )
    indices = details.groupby('trade', sort=False).apply(lambda df: df.index[-1])
    style = style.set_table_styles({i: [{'selector': '', 'props': 'border-bottom: 3px solid red;'}] for i in indices}, axis=1).set_caption('Details')
    style = style.hide(subset=details.columns[details.columns.str.startswith('_')], axis=1).set_sticky(axis=1).set_table_styles(table_styles)
    return style

In [14]:
def fn(
    pricer, 
    out1, 
    out2,
    event,
    send_email=True, 
    email_freq=1, 
    send_email_now=False, 
    do_one=False
):
    last_update = time.ctime()
    last_email_sent = time.time()
    p = pricer
    while True:
        if event.is_set():
            break
        try:
            reload(specs)
            from api.watchdog.specs import get_trades, get_targets
            trades = get_trades(clarion)
            targets = get_targets()
            p.get_data(runner, trades, targets, metrics=[*clarion_metrics.values()])
            clear_output()
            last_update = time.ctime()
            s1 = style_watch(p.watch)
            s2 = style_details(p.res)
            # clear_output()
            # out.clear_output(wait=True)
            out1.outputs = ()
            out1.append_display_data(last_update)
            out1.append_display_data(s1)
            out2.outputs = ()
            out2.append_display_data(last_update)
            out2.append_display_data(s2)
            # display(out)
            if send_email:
                if (((time.time() - last_email_sent) / 3600) >= email_freq) or send_email_now:
                    if send_email_now:
                        send_email_now = False
                    print('sending')
                    send_watch_email(s1, s2)
                    last_email_sent = time.time()
            # break
            if not do_one:
                time.sleep(30)
        except Exception as e:
            # return s1
            # raise e
            # clear_output()
            out1.outputs = ()
            out1.append_display_data(f"BROKEN: {last_update}")
            out1.append_display_data(s1)
            out2.outputs = ()
            out2.append_display_data(f"BROKEN: {last_update}")
            out2.append_display_data(s2)
            time.sleep(5)
        if do_one:
            break

In [15]:
p = Pricer()
out_summary = w.Output(layout={'overflow-x': 'auto'})
out_details = w.Output(layout={'overflow-x': 'auto', 'overflow-y': 'auto', 'height': '700px'})
event = threading.Event()

In [16]:
children = [out_summary, out_details]
tab = w.Tab(children=children, titles=['watchdog', 'details'])

In [17]:
thread = threading.Thread(target=fn, kwargs={
    'pricer': p, 
    'out1': out_summary, 
    'out2': out_details,
    'event': event,
    'send_email': True, # False,
    # 'send_email': False,
    'email_freq': 1,
    'send_email_now': True,
})

### WATCH

#### 

In [18]:
tab

In [19]:
thread.start()

In [ ]:
event.set()

### Send EMAIL

In [ ]:
ss = fn(
    p, 
    out_summary, 
    out_details,  
    event=threading.Event(),
    send_email=False, 
    send_email_now=False, 
    do_one=True
) 

In [ ]:
p.res.to_clipboard()

In [ ]:
p.get_data(runner, trades, targets, metrics=[*clarion_metrics.values()])

In [ ]:
s1 = style_watch(p.watch)
s2 = style_details(p.res)

In [ ]:
send_watch_email(s1, s2, test=True)

In [ ]:
p.res._repr_html_()

In [ ]:
from pandas.io import formats as fmt

In [ ]:
fmt.

In [ ]:
s1.to_html(doctype_html=True, notebook=False)

In [ ]:
s1.to_html(doctype_html=True)

In [ ]:
from premailer import transform

In [ ]:
styles = {
    "": "font-family: sans-serif; border-collapse: collapse;",
    "td, th": "border: 1px solid #ddd; text-align: left; padding: 8px;",
    "td": "white-space: nowrap"
}

In [ ]:
[{"selector": sel, 'props': '; '.join([': '.join(p) for p in props])} for sel, props in styles.items()]

In [ ]:
[{"selector": k, 'props': v} for k, v in styles.items()]

In [ ]:
styles = {
    "":[
      ('border', 'none'),
      ('border-collapse', 'collapse'),
      ('border-spacing', '0'),
      ('color', 'black'),
      ('font-size', '12px'),
      ('table-layout', 'fixed'),
      ('font-family', 'Calibri')
    ],
    'thead': [
      ('border-bottom', '1px solid #bdbdbd'),
      ('border-collapse', 'collapse'),
      ('vertical-align', 'bottom'),
      # ('font-family', 'Castellar')
    ],
    'tr, td': [
      ('text-align', 'left'),
      ('vertical-align', 'middle'),
      ('padding', '0.5em 0.5em'),
      ('line-height', 'normal'),
      ('white-space', 'nowrap'),
      ('max-width', 'none'),
      ('border', 'none'),
      #('border-collapse', 'collapse'),
    ],
    'th': [
        ('font-weight', 'bold'),
        ('text-align', 'right'),
        ('white-space', 'nowrap')
    ],
    'td': [
        ('white-space', 'nowrap')
    ],
    "tbody tr:nth-child(even)": [
      ('background', "#f5f5f5"),
    ],
    # 'tbody tr:hover': [
    #   ('background', 'rgba(66, 165, 245, 0.2)'),
    # ]
}

In [ ]:
st = [{"selector": sel, 'props': '; '.join([': '.join(p) for p in props])} for sel, props in styles.items()]

In [ ]:
st

In [ ]:
{"selector": "tbody tr:nth-child(even)","props":[("background-color","lightgrey")]}

In [ ]:
ss = s1.set_table_styles([{"selector": sel, 'props': '; '.join([': '.join(p) for p in props])} for sel, props in styles.items()])

In [ ]:
from premailer import Premailer
pm = Premailer(exclude_pseudoclasses=False)

In [ ]:
ss

In [ ]:
to_send = pm.transform(ss.to_html(doctype_html=True), pretty_print=True,)

In [ ]:
print(to_send)

In [ ]:
from_email = 'gioreports@brevanhoward.com'
smtp_server = "smtp.rivagecapital.com"
to_email = [
    'giovambattista.perciaccante@brevanhoward.com',
    # 'menashe.banit@brevanhoward.com'
    'gioxc88@gmail.com',
    'gioxc@hotmail.it',
    'omar.sindi@brevanhoward.com',
    'osindi@hotmail.com'
]

message = MIMEMultipart("alternative")
message["Subject"] = f'TRADE WATCH - {pd.Timestamp.now(): %d%b%y %H:%M}'
message["From"] = from_email
message["To"] = ','.join(to_email)


html_body = to_send
html_ = MIMEText(html_body, "html")
message.attach(html_)
smtp = smtplib.SMTP(smtp_server)
# smtp.set_debuglevel(1)
smtp.sendmail(from_email, to_email, message.as_string())

In [ ]:
pd.io.formats.style.Styler

In [ ]:
type(s1)

In [ ]:
send_watch_email(s1, s2)

### TEST

In [ ]:
p.watch

In [ ]:
r.df

In [ ]:
r = Runner(clarion)

In [ ]:
t = [
    clarion.instruments.RatesShortTermFuture(
        currency='USD',
        instrument_name='SOFR 3M CME',
        tenor='Z3',
        notional=1,
    ),
    clarion.instruments.RatesShortTermFuture(
        currency='USD',
        instrument_name='SOFR 3M CME',
        tenor='Z4',
        notional=-1.9,
    ),
    clarion.instruments.RatesShortTermFuture(
        currency='USD',
        instrument_name='SOFR 3M CME',
        tenor='Z5',
        notional=1,
    ),
    clarion.instruments.RatesShortTermFuture(
        currency='EUR',
        instrument_name='EURIBOR 3M ICE',
        tenor='M5',
        notional=1,
    ),
]

In [ ]:
r.run([*clarion_metrics.values()], t)

In [ ]:
r.df